In [1]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

from pathlib import Path
from tqdm import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
#from utils.post_processing import post_process


In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

In [3]:
CURRENT_DIR = Path.cwd()
FT_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning"
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

ERC_DIR = FT_DIR.parent
LLAMA_FACTORY_DIR = ERC_DIR / "LLaMA-Factory"

BASE_MODEL = "unsloth/llama-3-8b-Instruct-bnb-4bit"
LOGGING_DIR = FT_DIR / "training_logs"
OUTPUT_DIR = FT_DIR / "saved_models" / f"""comics35_pg_nb_xx_{BASE_MODEL.split("/")[1]}"""

In [4]:
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

In [5]:
train_dataset_name = f"""comics35_utterance_pg_classp_train.json"""
test_dataset_name = f"""comics35_utterance_pg_classp_test.json"""

train_dataset_file = DATASET_DIR / train_dataset_name
test_dataset_file = DATASET_DIR / test_dataset_name

In [6]:
train_dataset_file

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/datasets/comics35_utterance_pg_classp_train.json')

In [7]:

if not os.path.exists(os.path.join(FT_DIR, "model_args")):
    os.mkdir(os.path.join(FT_DIR, "model_args"))

train_file = FT_DIR / "model_args" / f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}.json"""

In [8]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [9]:
with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["comics"] = dataset_info_line

with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

In [10]:
NB_EPOCHS = 8

In [11]:
args = dict(
    
  stage="sft",                           # do supervised fine-tuning
  do_train=True,

  model_name_or_path=BASE_MODEL,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  num_train_epochs=NB_EPOCHS,            # the epochs of training
  output_dir=str(OUTPUT_DIR),                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents

  dataset="comics",                      # dataset name
  template="llama3",                     # use llama3 prompt template
  train_on_prompt=True,
  val_size=0.1,
  max_samples=10000,                       # use 500 examples in each dataset

  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  #temperature=0.5,
  
  warmup_ratio=0.1,                      # use warmup scheduler    
  learning_rate=5e-5,                    # the learning rate
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  
  fp16=True,                             # use float16 mixed precision training
  quantization_bit=4,                    # use 4-bit QLoRA  
  #use_liger_kernel=True,
  #quantization_device_map="auto",
  
  #load_best_model_at_end=True,
  #metric_for_best_model="eval_loss",
  #save_strategy="epoch",
  #eval_strategy="epoch",
  logging_steps=10,                      # log every 10 steps
  save_steps=5000,                       # save checkpoint every 1000 steps    
  logging_dir=str(LOGGING_DIR),
  
  #use_unsloth=True,
  report_to="tensorboard"                       # discards wandb

)

In [12]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [13]:
p = subprocess.Popen(["llamafactory-cli", "train", train_file], cwd=LLAMA_FACTORY_DIR)

In [14]:
p.wait()

11/14/2024 13:38:42 - INFO - llamafactory.cli - Initializing distributed tasks at: 127.0.0.1:27359


W1114 13:38:43.527000 140616479401280 torch/distributed/run.py:779] 
W1114 13:38:43.527000 140616479401280 torch/distributed/run.py:779] *****************************************
W1114 13:38:43.527000 140616479401280 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1114 13:38:43.527000 140616479401280 torch/distributed/run.py:779] *****************************************


11/14/2024 13:38:49 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/14/2024 13:38:49 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/14/2024 13:38:49 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, compute dtype: torch.float16
11/14/2024 13:38:49 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/14/2024 13:38:49 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/14/2024 13:38:49 - INFO - llamafactory.hparams.parser - Process rank: 1, device: cuda:1, n_gpu: 1, distributed training: True, compute dtype: torch.float16


[INFO|configuration_utils.py:672] 2024-11-14 13:38:49,277 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-14 13:38:49,277 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

11/14/2024 13:38:51 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>
11/14/2024 13:38:51 - INFO - llamafactory.data.loader - Loading dataset /Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/datasets/comics35_utterance_pg_classp_train.json...
11/14/2024 13:38:51 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


...

{'train_runtime': 897.8693, 'train_samples_per_second': 5.756, 'train_steps_per_second': 0.356, 'train_loss': 0.4043539797887206, 'epoch': 7.9}


[INFO|configuration_utils.py:672] 2024-11-14 13:53:58,503 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-14 13:53:58,504 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_co

***** train metrics *****
  epoch                    =      7.9012
  total_flos               = 133911409GF
  train_loss               =      0.4044
  train_runtime            =  0:14:57.86
  train_samples_per_second =       5.756
  train_steps_per_second   =       0.356


[INFO|modelcard.py:449] 2024-11-14 13:53:59,759 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


0

In [15]:
args = dict(
  model_name_or_path=BASE_MODEL, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=str(OUTPUT_DIR),            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)

In [16]:
model = ChatModel(args)

[INFO|configuration_utils.py:672] 2024-11-14 13:54:02,203 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-14 13:54:02,204 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

11/14/2024 13:54:03 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


[INFO|configuration_utils.py:672] 2024-11-14 13:54:03,899 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/config.json
[INFO|configuration_utils.py:739] 2024-11-14 13:54:03,901 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128255,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

11/14/2024 13:54:03 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.
11/14/2024 13:54:03 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
11/14/2024 13:54:03 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:400] 2024-11-14 13:54:04,027 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3726] 2024-11-14 13:54:04,358 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/f296897830363557c84cc4a942c2cd1f91818ae4/model.safetensors
[INFO|modeling_utils.py:1622] 2024-11-14 13:54:04,402 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1099] 2024-11-14 13:54:04,407 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "pad_token_id": 128255
}

[INFO|quantizer_bnb_4bit.py:122] 2024-11-14 13:54:04,577 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization
[INFO|modeling_utils.py:4568] 2024-11-14 13:

11/14/2024 13:54:07 - INFO - llamafactory.model.model_utils.attention - Using torch SDPA for faster training and inference.
11/14/2024 13:54:09 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_nb_xx_llama-3-8b-Instruct-bnb-4bit
11/14/2024 13:54:09 - INFO - llamafactory.model.loader - all params: 8,051,232,768


In [17]:
model.engine

In [18]:
model.engine_type

'huggingface'

In [19]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append(sample["instruction"] + sample["input"])
    #test_prompts.append(sample["input"])
    test_grounds.append(sample["output"])

In [20]:
test_predictions = []

In [21]:
messages = []

for prompt in tqdm(test_prompts, desc="Running inferences ..."):
    #print(type(prompt))
    #messages.append({"role": "user", "content": prompt})
    message = [{"role": "user", "content": prompt}]
    test_predictions.append(model.chat(message))
    #test_predictions.append(model.chat(prompt))

Running inferences ...:   6%|▌         | 9/156 [00:44<12:30,  5.11s/it]

[WARNING|logging.py:328] 2024-11-14 13:54:12,132 >> Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Running inferences ...: 100%|██████████| 156/156 [09:54<00:00,  3.81s/it]


In [22]:
# batch_size = 1  # Adjust batch size according to memory and model limitations

# # Initialize list to hold predictions
# test_predictions = []

# # Iterate over test_prompts in batches
# for i in tqdm(range(0, len(test_prompts), batch_size), desc="Running inferences ..."):
#     # Create a batch of messages
#     batch_prompts = test_prompts[i:i + batch_size]
    
#     # Prepare the messages for the batch
#     batch_messages = [[{"role": "user", "content": prompt}] for prompt in batch_prompts]
    
#     # Perform inference on the batch and store predictions
#     for message in batch_messages:
#         prediction = model.chat(message)
#         test_predictions.append(prediction)  # Collect batch predictions

In [23]:
len(test_predictions)

156

In [24]:
test_predictions

[[Response(response_text='{"page_utterance_emotions": [["surprise"], ["surprise"], ["surprise"], ["joy"], ["joy"], ["joy"], ["surprise"], ["joy"], ["joy"], ["neutral"], ["neutral"]]}', response_length=47, prompt_length=461, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["sadness"], ["sadness"], ["anger"], ["anger"], ["sadness"], ["fear", "sadness"], ["sadness"]]}', response_length=40, prompt_length=605, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["anger"], ["anger"], ["anger"], ["fear", "surprise"], ["anger"], ["fear"], ["anger"], ["anger"], ["anger"], ["joy"], ["anger"], ["anger"], ["anger"], ["joy"], ["joy"], ["joy"]]}', response_length=64, prompt_length=700, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "surprise"], ["anger", "surprise"], ["anger"], ["anger"], ["anger"], ["anger"], ["anger"], ["anger"], ["an

In [25]:
processed_preds = []
bad_idx = []

for i, raw_pred in enumerate(test_predictions):
    try:
        x = json.loads(raw_pred[0].response_text)["page_utterance_emotions"]
        processed_preds.append(x)
    except:
        print(i)
        bad_idx.append(i)

In [26]:
# bad_idx.sort(reverse=True)

# # Remove elements from 'grounds' at the specified indices
# for idx in bad_idx:
    
#     #del processed_grounds[idx]
#     del processed_preds[idx]

In [27]:
len(processed_preds)

156

In [28]:
processed_grounds = []

for ground in test_grounds:
    x = json.loads(ground)["page_utterance_emotions"]
    processed_grounds.append(x)

In [29]:
len(processed_grounds)

156

In [30]:
bad_idx

[]

In [31]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del processed_grounds[idx]
    #del processed_preds[idx]

In [32]:
bad_idx = []

for idx, (i,j) in enumerate(zip(processed_grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

14 23 24
27 17 18
85 13 14
103 13 12


In [33]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del processed_grounds[idx]
    del processed_preds[idx]

In [34]:
len(processed_grounds), len(processed_preds)

(152, 152)

In [35]:
grounds = [item for sublist in processed_grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [36]:
len(grounds), len(predictions)

(1260, 1260)

In [37]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [38]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['j', 'o', 'y'] will be ignored
  warnings.warn(


In [39]:
y_true_mhot.shape, y_pred_mhot.shape

((1260, 7), (1260, 7))

In [40]:
class_labels = mlb.classes_

In [41]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [42]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.609     0.577     0.593       430
     disgust      0.171     0.150     0.160        40
        fear      0.538     0.443     0.486       287
         joy      0.514     0.559     0.536       290
     neutral      0.407     0.250     0.310        96
     sadness      0.494     0.512     0.503       324
    surprise      0.568     0.500     0.532       342

   micro avg      0.535     0.500     0.517      1809
   macro avg      0.472     0.427     0.445      1809
weighted avg      0.534     0.500     0.514      1809
 samples avg      0.550     0.519     0.509      1809



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
# with open({Path(FT_DIR)} / "classification_report.pickle", 'wb') as fh:
    
#      pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)